<a href="https://colab.research.google.com/github/patero22/GNN-Reaserch_project/blob/main/Message_passing_and_peak_memory_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.2.1 torchvision torchaudio
!pip install torch-geometric
!pip install dgl==2.1.0
!pip install memory-profiler

In [2]:
#pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html

In [3]:
# import torch
# torch.utils.data.datapipes.utils.common.DILL_AVAILABLE = torch.utils._import_utils.dill_available()
# import torchdata

ImportError: cannot import name 'DILL_AVAILABLE' from 'torch.utils.data.datapipes.utils.common' (/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/utils/common.py)


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, PNAConv
from dgl.nn.pytorch import GraphConv, GATConv as GATConvDGL, SAGEConv as SAGEConvDGL

## Define Models (GCN, GAT, GraphSAGE, PNA)

### PyG Models

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, 8, heads=8)
        self.conv2 = GATConv(8*8, out_channels, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return x

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, 16)
        self.conv2 = SAGEConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class PNA(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(PNA, self).__init__()
        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']
        self.conv1 = PNAConv(in_channels, 16, aggregators, scalers)
        self.conv2 = PNAConv(16, out_channels, aggregators, scalers)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


### DGL Models

In [6]:
class GCN_DGL(torch.nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCN_DGL, self).__init__()
        self.conv1 = GraphConv(in_feats, 16)
        self.conv2 = GraphConv(16, out_feats)

    def forward(self, g, features):
        x = self.conv1(g, features)
        x = F.relu(x)
        x = self.conv2(g, x)
        return x

class GAT_DGL(torch.nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GAT_DGL, self).__init__()
        self.conv1 = GATConvDGL(in_feats, 8, num_heads=8)
        self.conv2 = GATConvDGL(8*8, out_feats, num_heads=1)

    def forward(self, g, features):
        x = self.conv1(g, features)
        x = F.elu(x)
        x = self.conv2(g, x)
        return x

class GraphSAGE_DGL(torch.nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GraphSAGE_DGL, self).__init__()
        self.conv1 = SAGEConvDGL(in_feats, 16, 'mean')
        self.conv2 = SAGEConvDGL(16, out_feats, 'mean')

    def forward(self, g, features):
        x = self.conv1(g, features)
        x = F.relu(x)
        x = self.conv2(g, x)
        return x

## Create Functions for COO and CSR Message Passing

For COO and CSR message passing, we'll leverage scipy for conversion and torch-sparse for COO operations.

In [7]:
from scipy.sparse import coo_matrix, csr_matrix

# Conversion Functions
def convert_to_coo(edge_index, num_nodes):
    row, col = edge_index
    data = torch.ones(row.size(0))
    coo = coo_matrix((data.numpy(), (row.numpy(), col.numpy())), shape=(num_nodes, num_nodes))
    return coo

def convert_to_csr(edge_index, num_nodes):
    row, col = edge_index
    data = torch.ones(row.size(0))
    csr = csr_matrix((data.numpy(), (row.numpy(), col.numpy())), shape=(num_nodes, num_nodes))
    return csr

## Define Profiling Function

In [8]:
# Profiling Function
from memory_profiler import memory_usage
import time

def profile_model(model, data, device, dgl=False, format='coo'):
    data = data.to(device)
    model = model.to(device)

    def forward_pass():
        if dgl:
            model(data, data.ndata['feat'])
        else:
            model(data)

    # Measure time
    start_time = time.time()
    for _ in range(100):
        forward_pass()
    end_time = time.time()

    # Measure peak memory usage
    mem_usage = memory_usage(forward_pass, interval=0.1)

    return (end_time - start_time) / 100, max(mem_usage)

## Load Datasets (Karate Club and Citeseer)

In [9]:
from torch_geometric.datasets import KarateClub, Planetoid
from dgl.data import KarateClubDataset, CiteseerGraphDataset

In [10]:
# Load PyG Datasets
dataset_karate_pyg = KarateClub()
data_karate_pyg = dataset_karate_pyg[0]

dataset_citeseer_pyg = Planetoid(root='data/Citeseer', name='Citeseer')
data_citeseer_pyg = dataset_citeseer_pyg[0]

In [11]:
# Load DGL Datasets
def load_karate_dgl():
    dataset = KarateClubDataset()
    graph = dataset[0]
    return graph

def load_citeseer_dgl():
    dataset = CiteseerGraphDataset()
    graph = dataset[0]
    return graph

graph_karate_dgl = load_karate_dgl()
graph_citeseer_dgl = load_citeseer_dgl()

  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


## Test Models with COO and CSR on CPU and GPU

### Define devices

In [12]:
device_cpu = torch.device('cpu')
device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Test GCN on CPU and GPU for PyG Karate ClubB

In [13]:
gcn_model_pyg = GCN(dataset_karate_pyg.num_features, dataset_karate_pyg.num_classes)
time_cpu, mem_cpu = profile_model(gcn_model_pyg, data_karate_pyg, device_cpu, format='coo')
time_gpu, mem_gpu = profile_model(gcn_model_pyg, data_karate_pyg, device_gpu, format='coo')
print(f'PyG Karate Club GCN on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
print(f'PyG Karate Club GCN on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')


PyG Karate Club GCN on CPU: 0.005395 seconds per iteration, 728.23 MB peak memory
PyG Karate Club GCN on GPU: 0.002583 seconds per iteration, 728.40 MB peak memory


### Test GCN on CPU and GPU for PyG Citeseer

In [14]:
gcn_model_pyg = GCN(dataset_citeseer_pyg.num_features, dataset_citeseer_pyg.num_classes)
time_cpu, mem_cpu = profile_model(gcn_model_pyg, data_citeseer_pyg, device_cpu, format='csr')
time_gpu, mem_gpu = profile_model(gcn_model_pyg, data_citeseer_pyg, device_gpu, format='csr')
print(f'PyG Citeseer GCN on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
print(f'PyG Citeseer GCN on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')


PyG Citeseer GCN on CPU: 0.037767 seconds per iteration, 732.36 MB peak memory
PyG Citeseer GCN on GPU: 0.049320 seconds per iteration, 732.62 MB peak memory


### Test GCN on CPU and GPU for DGL Karate Club

`feat` error solver -v

In [15]:
# Load the Karate Club dataset
dataset_karate_dgl = KarateClubDataset()
graph_karate_dgl = dataset_karate_dgl[0]

# Add dummy features (e.g., use one-hot encoding for simplicity)
num_nodes = graph_karate_dgl.num_nodes()
graph_karate_dgl.ndata['feat'] = torch.eye(num_nodes)

# Add labels
graph_karate_dgl.ndata['label'] = torch.tensor(graph_karate_dgl.ndata['label'])


<ipython-input-15-5a57f9503770>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  graph_karate_dgl.ndata['label'] = torch.tensor(graph_karate_dgl.ndata['label'])


In [16]:
gcn_model_dgl = GCN_DGL(graph_karate_dgl.ndata['feat'].shape[1], graph_karate_dgl.ndata['label'].max().item() + 1)
time_cpu, mem_cpu = profile_model(gcn_model_dgl, graph_karate_dgl, device_cpu, dgl=True, format='coo')
time_gpu, mem_gpu = profile_model(gcn_model_dgl, graph_karate_dgl, device_gpu, dgl=True, format='coo')
print(f'DGL Karate Club GCN on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
print(f'DGL Karate Club GCN on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')


DGL Karate Club GCN on CPU: 0.004457 seconds per iteration, 735.27 MB peak memory
DGL Karate Club GCN on GPU: 0.005188 seconds per iteration, 735.27 MB peak memory


### Test GCN on CPU and GPU for DGL Citeseer

In [17]:
gcn_model_dgl = GCN_DGL(graph_citeseer_dgl.ndata['feat'].shape[1], graph_citeseer_dgl.ndata['label'].max().item() + 1)
time_cpu, mem_cpu = profile_model(gcn_model_dgl, graph_citeseer_dgl, device_cpu, dgl=True, format='csr')
time_gpu, mem_gpu = profile_model(gcn_model_dgl, graph_citeseer_dgl, device_gpu, dgl=True, format='csr')
print(f'DGL Citeseer GCN on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
print(f'DGL Citeseer GCN on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')

DGL Citeseer GCN on CPU: 0.116792 seconds per iteration, 782.54 MB peak memory
DGL Citeseer GCN on GPU: 0.057592 seconds per iteration, 782.58 MB peak memory


# More coplex datasets

### Loading Complex Datasets and Profiling
Define Profiling Function

In [18]:
def profile_model(model, data, device, dgl=False, format='coo'):
    model = model.to(device)
    data = data.to(device)

    def run_model():
        model.train()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        for _ in range(1):  # Run for one epoch to measure time
            optimizer.zero_grad()
            if dgl:
                if format == 'csr':
                    logits = model(data, data.ndata['feat'])
                else:
                    logits = model(data, data.ndata['feat'])
            else:
                logits = model(data)
            loss = F.cross_entropy(logits[data.y_index], data.y)
            loss.backward()
            optimizer.step()

    # Measure time
    start_time = time.time()
    run_model()
    end_time = time.time()
    time_taken = end_time - start_time

    # Measure memory
    mem_usage = memory_usage(run_model, max_usage=True, retval=False)
    peak_memory = max(mem_usage)

    return time_taken, peak_memory

### Load Complex Datasets

#### Load PyG Datasets

In [19]:
from torch_geometric.datasets import Reddit, Amazon

In [20]:
dataset_reddit_pyg = Reddit(root='/tmp/Reddit')
data_reddit_pyg = dataset_reddit_pyg[0]

dataset_amazon_pyg = Amazon(root='/tmp/Amazon', name='Computers')
data_amazon_pyg = dataset_amazon_pyg[0]

KeyboardInterrupt: 

#### Load DGL Datasets

In [ ]:
from dgl.data import RedditDataset, AmazonCoBuyComputerDataset

In [ ]:
def load_reddit_dgl():
    dataset = RedditDataset()
    graph = dataset[0]
    return graph

def load_amazon_dgl():
    dataset = AmazonCoBuyComputerDataset()
    graph = dataset[0]
    return graph

graph_reddit_dgl = load_reddit_dgl()
graph_amazon_dgl = load_amazon_dgl()

### Test Models on Complex Datasets

In [ ]:
# Define devices
device_cpu = torch.device('cpu')
device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Define and test models

In [ ]:
models = {
    'GCN': (GCN, GCN_DGL),
    'GAT': (GAT, GATConvDGL),
    'GraphSAGE': (GraphSAGE, SAGEConvDGL),
    'PNA': (PNA, PNAConv)
}

datasets_pyg = {
    'Reddit': data_reddit_pyg,
    'Amazon': data_amazon_pyg
}

datasets_dgl = {
    'Reddit': graph_reddit_dgl,
    'Amazon': graph_amazon_dgl
}

In [ ]:
for model_name, (ModelPyG, ModelDGL) in models.items():
    for dataset_name, data_pyg in datasets_pyg.items():
        # PyG Models
        model_pyg = ModelPyG(data_pyg.num_features, data_pyg.num_classes)
        time_cpu, mem_cpu = profile_model(model_pyg, data_pyg, device_cpu, format='coo')
        time_gpu, mem_gpu = profile_model(model_pyg, data_pyg, device_gpu, format='coo')
        print(f'PyG {dataset_name} {model_name} on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
        print(f'PyG {dataset_name} {model_name} on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')

    for dataset_name, graph_dgl in datasets_dgl.items():
        # DGL Models
        model_dgl = ModelDGL(graph_dgl.ndata['feat'].shape[1], graph_dgl.ndata['label'].max().item() + 1)
        time_cpu, mem_cpu = profile_model(model_dgl, graph_dgl, device_cpu, dgl=True, format='coo')
        time_gpu, mem_gpu = profile_model(model_dgl, graph_dgl, device_gpu, dgl=True, format='coo')
        print(f'DGL {dataset_name} {model_name} on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
        print(f'DGL {dataset_name} {model_name} on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')

ERROR : Your session crashed after using all available RAM

# Intermediate Complexity Datasets

## Load Intermediate Complexity Datasets

In [21]:
from torch_geometric.datasets import Planetoid
from dgl.data import CoraGraphDataset, PubmedGraphDataset, CiteseerGraphDataset

# Load PyG Datasets
dataset_cora_pyg = Planetoid(root='/tmp/Cora', name='Cora')
data_cora_pyg = dataset_cora_pyg[0]

dataset_pubmed_pyg = Planetoid(root='/tmp/Pubmed', name='Pubmed')
data_pubmed_pyg = dataset_pubmed_pyg[0]

dataset_citeseer_pyg = Planetoid(root='/tmp/Citeseer', name='Citeseer')
data_citeseer_pyg = dataset_citeseer_pyg[0]

# Load DGL Datasets
def load_cora_dgl():
    dataset = CoraGraphDataset()
    graph = dataset[0]
    return graph

def load_pubmed_dgl():
    dataset = PubmedGraphDataset()
    graph = dataset[0]
    return graph

def load_citeseer_dgl():
    dataset = CiteseerGraphDataset()
    graph = dataset[0]
    return graph

graph_cora_dgl = load_cora_dgl()
graph_pubmed_dgl = load_pubmed_dgl()
graph_citeseer_dgl = load_citeseer_dgl()


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


## Test Models on Intermediate Datasets

In [22]:
# Define devices
device_cpu = torch.device('cpu')
device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define and test models
models = {
    'GCN': (GCN, GCN_DGL),
    'GAT': (GAT, GATConvDGL),
    'GraphSAGE': (GraphSAGE, SAGEConvDGL),
    'PNA': (PNA, PNAConv)
}

datasets_pyg = {
    'Cora': data_cora_pyg,
    'PubMed': data_pubmed_pyg,
    'CiteSeer': data_citeseer_pyg
}

datasets_dgl = {
    'Cora': graph_cora_dgl,
    'PubMed': graph_pubmed_dgl,
    'CiteSeer': graph_citeseer_dgl
}

for model_name, (ModelPyG, ModelDGL) in models.items():
    for dataset_name, data_pyg in datasets_pyg.items():
        # PyG Models
        model_pyg = ModelPyG(data_pyg.num_features, data_pyg.num_classes)
        time_cpu, mem_cpu = profile_model(model_pyg, data_pyg, device_cpu, format='coo')
        time_gpu, mem_gpu = profile_model(model_pyg, data_pyg, device_gpu, format='coo')
        print(f'PyG {dataset_name} {model_name} on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
        print(f'PyG {dataset_name} {model_name} on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')

    for dataset_name, graph_dgl in datasets_dgl.items():
        # DGL Models
        model_dgl = ModelDGL(graph_dgl.ndata['feat'].shape[1], graph_dgl.ndata['label'].max().item() + 1)
        time_cpu, mem_cpu = profile_model(model_dgl, graph_dgl, device_cpu, dgl=True, format='coo')
        time_gpu, mem_gpu = profile_model(model_dgl, graph_dgl, device_gpu, dgl=True, format='coo')
        print(f'DGL {dataset_name} {model_name} on CPU: {time_cpu:.6f} seconds per iteration, {mem_cpu:.2f} MB peak memory')
        print(f'DGL {dataset_name} {model_name} on GPU: {time_gpu:.6f} seconds per iteration, {mem_gpu:.2f} MB peak memory')


AttributeError: 'GlobalStorage' object has no attribute 'num_classes'